# pdf2reveal
This notebook transforms a presentation PDF made using LibreOffice Impress, Microsoft Powerpoint or Latex Beamer into HTML (reveal.js). This allows displaying the presentation on any device running a web browser. 

I wrote this so that I can present lecture slides on a Promethean without having to connect my computer. 

### Technical details

###### Initialization

During the initial run of this notebook, the **install_reveal** function installed the latest version of **reveal.js** which was **4.1.3**. The **create_template** function then created a jinja2 template by editing 
**reveal.js-master/index.html**. Note that my notebook code may break if a later version of **reveal.js** is installed.

###### GitHub repository
A copy of this notebook and all files and directories created by it is maintained in a GitHub repository at https://github.com/aubreymoore/albi345-slides. This repo serves GitHub pages at https://aubreymoore.github.io/albi345-slides/. 

###### Example usage
```
papermill \ 
-p PDFPATH /home/aubrey/ALBI-345/files/pdfs/Orders-with-Complete-Metamorphosis.pdf \ 
pdf2reveal.ipynb \ 
out.ipynb
```
After running this example and pushing the local repo to gitHub, the online HTML (reveal.js) presentation will be available at 
https://aubreymoore.github.io/albi345-slides/Orders-with-Complete-Metamorphosis

###### Directory structure:
```
pdf2reveal.ipynb          This notebook.
template.html             Created during intitial run of this notebook.
out.ipynb                 Created when running this notebook using papermill. Useful for error checking.
reveal.js-master (dir)    Created during intitial run of this notebook.
images (dir)              Created during intitial run of this notebook.
   presentation1-000.jpg  Created when PDFPATH is "*/presentation1.pdf"
   presentation1-001.jpg  Created when PDFPATH is "*/presentation1.pdf"
   ...
   presentation2-000.jpg  Created when PDFPATH is "*/presentation2.pdf"
   presentation2-001.jpg  Created when PDFPATH is "*/presentation2.pdf"
   ...
presentation1 (dir)       Created when PDFPATH is "*/presentation1.pdf"
   presentation1.pdf      Created when PDFPATH is "*/presentation1.pdf" Copy of original PDF.
   index.html             Created when PDFPATH is "*/presentation1.pdf"   
presentation2 (dir)       Created when PDFPATH is "*/presentation2.pdf"
   presentation2.pdf      Created when PDFPATH is "*/presentation2.pdf" Copy of original PDF.
   index.html             Created when PDFPATH is "*/presentation2.pdf"
```

##### Template

During the initial first run of this notebook
The create_template function 
reveal.js 4.1.3

##### Fixing imagemagick problems
This notebook uses the **convert** command, which is part of **imagemagick** to convert PDFs to images.
Had some initial problems which were fixed by changing a few lines in the config file **/etc/ImageMagick-6/policy.xml**:
```
<policy domain="coder" rights="none" pattern="PDF" />
<policy domain="resource" name="memory" value="256MiB"/>
<policy domain="resource" name="disk" value="1GiB"/>
```
where changed to:
```
<!--  <policy domain="coder" rights="none" pattern="PDF" />  -->
<policy domain="resource" name="memory" value="8GiB"/>
<policy domain="resource" name="disk" value="8GiB"/>
```
### References
* https://marcus-baw.medium.com/using-reveal-js-a74b30e4065b
* https://martinomensio.medium.com/how-to-host-reveal-js-slides-on-github-pages-and-have-a-tidy-repository-1a363944c38d
* https://superuser.com/questions/1178666/imagemagick-convert-quits-after-some-pages

In [1]:
import os
import requests, zipfile, io
import jinja2
import glob
import shutil

In [2]:
# Parameters which may be set using papermill
PDFPATH = '/home/aubrey/Downloads/applied-ent.pdf'

In [3]:
# Parameters
PDFPATH = "/home/aubrey/ALBI-345/files/pdfs/Orders-with-Incomplete-Metamorphosis.pdf"


In [4]:
def install_reveal():
    """
    Download and install the latest version of reveal.js.
    """
    if not os.path.exists('reveal.js-master'):
        r = requests.get('https://github.com/hakimel/reveal.js/archive/master.zip')
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall()

In [5]:
def create_template():
    """
    Create a jinja2 template by editing the reveal.js default index.html
    """

    # Read the default index.html as a string    
    s = open('reveal.js-master/index.html', 'r').read()

    # edit the string and save it as template.html
    s = s.replace('"dist', '"../reveal.js-master/dist')
    s = s.replace('"plugin', '"../reveal.js-master/plugin')
    s = s.replace('<section>Slide 1</section>\n\t\t\t\t<section>Slide 2</section>',
              '{% for image in images %}\n\t\t\t\t\t<section> <img src="../{{ image }}"> </section>\n\t\t\t\t{% endfor %}')

    with open('template.html', 'w') as f:
        f.write(s)

In [6]:
def create_presentation():
    """
    Use the jinja2 template to add slides from the images directory
    """
    
    if not os.path.exists('images'):
        os.mkdir('images')    
    
    # Create a directory for the presentation and put a copy of the PDF in it
    pdf_name = os.path.basename(PDFPATH).replace('.pdf', '')
    os.mkdir(pdf_name)
    shutil.copyfile(PDFPATH, f'{pdf_name}/{pdf_name}.pdf')
    
    # Convert the PDF pages into jpg images and store them in the communal images directory
    os.system(f'convert -density 300 {PDFPATH} images/{pdf_name}-%03d.jpg')
    images = sorted(glob.glob(f'images/{pdf_name}-???.jpg'))  
    print(images)
       
    templateLoader = jinja2.FileSystemLoader(searchpath="./")
    templateEnv = jinja2.Environment(loader=templateLoader)
    TEMPLATE_FILE = "template.html"
    template = templateEnv.get_template(TEMPLATE_FILE)
    outputText = template.render({'images': images})

    with open(f'{pdf_name}/index.html', 'w') as out:
        out.write(outputText)

In [7]:
# MAIN

if not os.path.exists('reveal.js-master'):
    install_reveal()
    create_template()
create_presentation()
print(f'Finished converting {PDFPATH}')
print('Do not forget to update git and push to GitHub.')

['images/Orders-with-Incomplete-Metamorphosis-000.jpg', 'images/Orders-with-Incomplete-Metamorphosis-001.jpg', 'images/Orders-with-Incomplete-Metamorphosis-002.jpg', 'images/Orders-with-Incomplete-Metamorphosis-003.jpg', 'images/Orders-with-Incomplete-Metamorphosis-004.jpg', 'images/Orders-with-Incomplete-Metamorphosis-005.jpg', 'images/Orders-with-Incomplete-Metamorphosis-006.jpg', 'images/Orders-with-Incomplete-Metamorphosis-007.jpg', 'images/Orders-with-Incomplete-Metamorphosis-008.jpg', 'images/Orders-with-Incomplete-Metamorphosis-009.jpg', 'images/Orders-with-Incomplete-Metamorphosis-010.jpg', 'images/Orders-with-Incomplete-Metamorphosis-011.jpg', 'images/Orders-with-Incomplete-Metamorphosis-012.jpg', 'images/Orders-with-Incomplete-Metamorphosis-013.jpg', 'images/Orders-with-Incomplete-Metamorphosis-014.jpg', 'images/Orders-with-Incomplete-Metamorphosis-015.jpg', 'images/Orders-with-Incomplete-Metamorphosis-016.jpg', 'images/Orders-with-Incomplete-Metamorphosis-017.jpg', 'images/O